# CS-5600/6600 Lecture 21 - Convolutional Neural Networks
**Dylan Zwick**

*Weber State University*

<center>
  <img src="https://imgs.xkcd.com/comics/trained_a_neural_net.png" alt="XKCD Trained a Neural Network">
</center>

In this lecture we'll discuss the basic concepts behind a "convolutional neural network", or [CNN](https://youtu.be/i739Gx2q6Ak?si=TL22jYFAikgsB2XB), and how they can be used to significantly improve performance when there are certain types of structure to the data. In particular, CNNs have been extremely useful when dealing with images, and are one of the most important machine learning tools in the field of computer vision.

A CNN doesn't fundamentally alter the methodology of the neural network - instead, it modifies its structure to address an underlying structure in the data - specifically, the relative importance of data that is close together.

**The Problem With Images**

In our last lecture, we created a basic sequential neural network to learn how to classify hand-written digits - and it did quite well! However, while it's not a trivial problem, it's much, *much* simpler than most computer vision problems.

One reason for this is the size of the data for each observation. For the hand-written digits, we had a $28 \times 28$ grid, and each grid had a greyscale score between 0 and 255, which meant 784 inputs per observation. That's not a trivial number, but it's much smaller than the amount of data in a standard digital image. For a standard digital image something like a $200 \times 200$ grid is much more common, and instead of there being a single greyscale value, there are typically 3 (RGB) color values, which would mean each observation would have about $200 \times 200 \times 3 = 120,000$ data points. That's a lot of data, and consequently a lot of weights to learn! With that many weights, convergence can be slow, and you need *a lot* of data if you want to avoid overfitting.

So, what does a convolutional neural network do? It looks for patterns by applying "convolution filters" to the images, and then learns to classify the images based upon the information from these filters - which is usually a much smaller amount of information!

**Convolution Filters**

What is a convolution filter? Let's look at it in the context of an exceptionally easy image classification problem - differentiating Xs and Os. So, instead of trying to learn 10 digits, here we're just learning two characters. Further, let's suppose we've only got a $9 \times 9$ grid and that each pixel in the grid is a bitmap, which means it's either a $0$ or a $1$. Here we could probably train just a simple perceptron (logistic regression) model and it would do a decent job.

<center>
  <img src="https://drive.google.com/uc?export=view&id=1ZcnqlMt5Y_EcKdOpXbEdshmhZ5YnPv_v" alt="Xs and Os">
</center>

However, this example highlights some important aspects about how we might use a convolutional neural network. For example, there are features in the data that we'd expect to see for an "X" that we would not expect to see for an "O". We wouldn't expect to see a hard left-to-right or right-to-left diagonal line on an "O", but we'd definitely expect to see one on an "X". Even more, we wouldn't expect to see a crossed line on an "O", but we'd absolutely expect to find one on an "X". Conversely, on an "O" we'd expect to see curves and vertical / horizontal lines.

The important thing here is that these features that are critical for distinguishing between an "X" and an "O" relate to pixels that are *near* each other. In other words, the interactions between inputs that are - in some sense - close to each other are important, and it would be nice if this were reflected within our model.

This is the idea behind a convolution filter, which is the foundation of a convolutional neural network. What a convolution filter does it it takes a pattern of interest (like the left to right line below), and moves over the data attempting to determine whether and where that pattern occurs. Generally speaking, in mathematics a "convolution" is some measure of how similar two things are to each other. They come up *all the time* in things like signal processing.

<center>
  <img src="https://drive.google.com/uc?export=view&id=1bEIYIFYg-1HLoPuRS8DTdv3CYn0K1HLM" alt="Convolution Filter Overview">
</center>

The way it does this is by taking the "convolution" of the pattern with a segment of the data.

For example, we could format our data is such a way that a black pixel is represented by a 1, and a white pixel is represented by a -1. Then, we could do a pixel-wise multiplication of our pattern with a segment of our image, and then calculate the average of this multiplication. This will give us a number between -1 and 1. A 1 would incidate perfect alignment with our pattern, while a -1 would indicate an exact misalignment.

<center>
  <img src="https://drive.google.com/uc?export=view&id=1QK4NUa_U5WfHSA9-A_qdbIhPicdnVlaw" alt="Convolution Filter Example">
</center>

We then move through the image, calculating this convolution on various segments, and forming an array of convolutions.

<center>
  <img src="https://drive.google.com/uc?export=view&id=1t0RFiG8Ea7WJUxWlc3UDPIPNfga4_S-r" alt="Convolution Array">
</center>
    
This array can then inform us as to which segments of the image correspond with our pattern, and which do not. We then repeat this for the various patterns of interest for our problem.

<center>
  <img src="https://drive.google.com/uc?export=view&id=1V2YcvbSunVyGEmE-hIS69uUrrudnviM5" alt="Convolution Pattern">
</center>

These convolutions are then sent to the *convolution layer* of our neural network. The convolution layer is the main building block of a CNN, and we note that it's a restricted layer - the nodes from the previous layer are only connected to a subset of nodes within the convolution layer! This can significantly decrease the number of weights in the neural network, which can decrease the variance and increase the training speed. Big win!

A few important things to note about CNNs:

* Generally speaking the filters are *learned*, not prescribed as in the example above. This is one of the things that's so cool about machine learning - with enough data and the right setup, the program will actually learn which patterns are important and look for those.
* The approach is only applicable for certain types of data. Specifically, data where there is a connection between adjacent or near entries.

<center>
  <img src="https://drive.google.com/uc?export=view&id=1Q1ub3GWi4w6i2wR41xU0GN9Xk6LEz0Xf" alt="Convolution Pattern">
</center>

**Implementing CNNs in Keras**

We'll now see how we can use convolutional neural networks in Keras, and show how they can be used to improve performance even on our old favorite MNIST dataset. But first, we'll import our favorite libraries, including our new favorites.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

Now, we'll grab the MNIST dataset from Keras.

In [ ]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

In previous lectures, we've used a Sequential neural network model. Such models are easy to use, but their applicability is quite limited: it can only express models with a single input and a single output, applying one layer after the other in a sequential fashion. In practice, it's pretty common to encounter models with multiple inputs (say, an image and its metadata), multiple outputs (different things you want to predict about the data), or a nonlinear topology (the "shape" of the neural network).

In cases where a linear model is insufficient, models are build in Keras with the Functional API. This is what most Keras models in the "wild" use. Here's a convolutional neural network. We'll first talk about the Functional API aspects of it, and then the actual convolutional layers themselves.

In [ ]:
inputs = keras.Input(shape=(28,28,1), name="Handwritten Digits")
x = keras.layers.Conv2D(filters=32,kernel_size=3,activation="relu") (inputs)
x = keras.layers.MaxPooling2D(pool_size = 2)(x)
x = keras.layers.Conv2D(filters = 64, kernel_size=3, activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs = inputs, outputs = outputs)

Let's go over this step by step.

We started by declacing an *Input*. Note that we can give names to these inputs, but it's optional. The inputs object holds information about the shape and possibly the dtype of the data that the model will process.

In [ ]:
inputs.shape

Here "None" just means we haven't specified how much data we'll be using in our training set. That's fine.

In [ ]:
inputs.dtype

Next, we create a layer, "x", and call it on the input. We then fed this through a sequence of new layers, always calling "x" upon itself.

Finally, we defined the nature of our output. We then instantiated our model by calling the *Model* function in Keras with the specified inputs and outputs. The outputs encode all the layers that create them. We can check out a summary of our model:

In [ ]:
model.summary()

In total, we have a svelte 104,202 parameters. Do you know what these types of models are called? They're called "non-parametric" models.

Alright, now let's go through the various layers.

In the MNIST dataset, the first layer of the model takes a feature map of size $(28, 28, 1)$, and outputs a feature map of size $(26, 26, 32)$. The number $32$ here is specified, and is the number of filter that we learn for that first layer. So, what's produced from this first 2D convolution layer is a feature map of size $(26,26,32)$ - the 32 filters applied to each $3 \times 3$ grid in the image. Note that we don't specify what these filters are - that's part of the learning process.

OK, so what about that *max_pooling2D* function? What's that? Well, what it does is it takes each $2 \times 2$ subgrid, and it produces the maximum value observed there for each feature. In other words, it aggressively downsamples our feature maps.

Why would we want to do this? Well, let's take a look at what would happen if we didn't have these pooling layers:

In [ ]:
inputs = keras.Input(shape=(28,28,1), name="Handwritten Digits")
x = keras.layers.Conv2D(filters=32,kernel_size=3,activation="relu") (inputs)
x = keras.layers.Conv2D(filters = 64, kernel_size=3, activation="relu")(x)
x = keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(10, activation="softmax")(x)
model_without_pooling = keras.Model(inputs = inputs, outputs = outputs)

We can get a summary of this model:

In [ ]:
model_without_pooling.summary()

What are some issues with this model?

* We want to make sure that, after a few layers, the convolution maps from disparate regions of our image are interacting with each other. If we only had three Conv2D layers then the $3 \times 3$ windows in the third layer would only contain information coming from $7 \times 7$ windows in the initial input. The high-level patterns learned will still be very small with regard to the initial input, which may not be enough to learn what we need to learn. So, why not just put in enough convolution layers to fully shrink down our image? Well, that leads to our second point.

* If we only had 3 convolution layers, then the final  feature map would have 712,202 parameters. That's a lot! Unless you have a lot of data, that's way too many parameters, and you'll get intense overfitting. So, those pooling layers help decrease the overall size of your model, which makes it faster to train and more robust (less prone to overfitting). Please note that if we added a bunch more convolution layers, we'd have a ridiculously large number of parameters - much more than every pixel in our training data.

Alright, let's see how these models actually perform:

In [ ]:
model_without_pooling.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_without_pooling.fit(train_images, train_labels, epochs=5, batch_size = 64)

That's a pretty great accuracy! On... the training data. Let's see how it does on the test data.

In [ ]:
model_without_pooling.evaluate(test_images, test_labels)

In [ ]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size = 64)

Similarly great accuracy! Let's check out how it does on the test data.

In [ ]:
model.evaluate(test_images, test_labels)

Look at that! It does a little better, plus it trains a lot faster.

##References

- 3Blue1Brown Video on [Convolutions](https://youtu.be/KuXjwB4LzSA?si=Xv0O89N81l4JK8Y0)
* 3Blue1Brown Video Series on [Deep Learning](https://www.youtube.com/watch?v=aircAruvnKk&t=4s)